---
title: Time resolution effect
abstract: To understand if the difference of Juno distribution at 1AU and beyond is due to time resolution, i.e. it's possile that 1s is sufficient to resolve distribution at r>1.5AU, but insufficient at 1AU. To check this we may compare ARTEMIS data at 1AU for two resolutions - 1s and 1/5s
---

## Process Wind data

In [2]:
%load_ext autoreload
%autoreload 2

In [2]:
from space_analysis.utils.speasy import Variables
from discontinuitypy.datasets import IDsDataset
from discontinuitypy.utils.basic import resample
from beforerr.polars import pl_norm

from datetime import timedelta
from space_analysis.plasma.formulary import df_thermal_spd2temp

In [ ]:
ts = timedelta(seconds=1 / 11)
tau = timedelta(seconds=60)

In [4]:
timerange = ['2016-01-01', '2016-06-29']
wind_mag_vars = Variables(
    provider = 'archive/local',
    dataset="WI_H4-RTN_MFI",
    parameters=["BRTN"],
    timerange=timerange
)

wind_plasma_vars = Variables(
    dataset="WI_K0_SWE",
    parameters=["Np", "V_GSM", "THERMAL_SPD"],
    timerange=timerange,
).retrieve_data()

In [ ]:
wind_density_col = wind_plasma_vars.data[0].columns[0]
wind_vec_cols = wind_plasma_vars.data[1].columns
wind_temperature_col = wind_plasma_vars.data[2].columns[0]

In [8]:
wind_mag_data = wind_mag_vars.to_polars()

wind_plasma_data = (
    wind_plasma_vars.to_polars()
    .with_columns(
        plasma_speed=pl_norm(wind_vec_cols),
    )
    .pipe(df_thermal_spd2temp, wind_temperature_col)
    .rename(
        {
            wind_density_col: "plasma_density",
        }
    )
)

In [10]:
# for freq in [11, 5 , 2, 1, 0.5]:
for freq in [5]:
    ts = timedelta(seconds=1 / freq)

    wind_ids_dataset = (
        IDsDataset(
            mag_data=wind_mag_data.pipe(resample, every=ts),
            plasma_data=wind_plasma_data,
            tau=tau,
            ts=ts,
            vec_cols=wind_vec_cols,
            density_col="plasma_density",
            temperature_col="plasma_temperature",
        )
        .find_events(return_best_fit=False)
        .update_candidates_with_plasma_data()
        .export(f"../data/05_reporting/events.Wind.ts_{1/freq:.2f}s_tau_60s.arrow")
    )

08-Feb-24 13:47:14: UserWarning: Distributing <class 'pandas.core.frame.DataFrame'> object. This may take some time.



Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 15:   0%           Elapsed time: 00:00, estimated remaining time: ?